In [ ]:
using Fluidum
using MaximumEntropy

In [ ]:
#define the grid
rmax = 30;
gridpoints=400;
grid_params = Fluidum.GridParameters(rmax, gridpoints);

#define the detector to use 
det = :ALICE;
det_name = detector_dict[det].name; 
dσ_QQdy = detector_dict[det].dσ_QQdy;
σ_in = detector_dict[det].σ_in;

#centrality classes
cent1 = 0;
cent2 = 10;

#define the initial parameters
tspan = (0.4,12);
Norm = 90.; 
tau0 = tspan[1];
tau_fs = 0.1;
rdrop = 8.5;
initial_params = Fluidum.InitialParameters(Norm, tau0, tau_fs, rdrop);

In [ ]:

#take the tabulated data 
ent = TabulatedData(string(@__DIR__, "\\EntropyProfile_", det_name, ".txt"));
ncoll = TabulatedData(string(@__DIR__, "\\NcollProfile_", det_name, ".txt"));
fonll = TabulatedData(string(@__DIR__, "\\FONLL_", det_name, ".txt"));


In [ ]:
fields, ccbar = initialize_fields(ent, ncoll, fonll, cent1, cent2; grid_params, initial_params, dσ_QQdy, m=1.5);
eos = Heavy_Quark(readresonancelist(), ccbar)


In [ ]:
viscosity = QGPViscosity(0.1,0.2); #or, ZeroViscosity();
bulk = SimpleBulkViscosity(0.083,15.0); #or, ZeroBulkViscosity();

diffusion_Ds = HQdiffusion(0.1, 1.5);
params_Ds=Fluidum.FluidProperties(eos,viscosity,bulk,diffusion_Ds);

In [ ]:
fo_Ds = freeze_out_routine(fields.discrete_field, Fluidum.matrxi1d_visc_HQ!, params_Ds, fields.initial_field, tspan; Tfo = 0.156);
field_results_Ds = oneshoot(fields.discrete_field, Fluidum.matrxi1d_visc_HQ!, params, fields.initial_field, tspan; reltol = 1e-8);


In [ ]:

plot_fields_evolution(field_results_Ds, fields.discretization, tspan, params_Ds)
